In [1]:
import rasterio
import affine
from rasterio import plot
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
IMAGE_PIXEL = 90
IMAGE_WIDTH = 122

In [2]:
imagePath = '../Données/S2A_MSIL2A_20190227T030651_N0211_R075_T48PYU_20190227T083207/S2A_MSIL2A_20190227T030651_N0211_R075_T48PYU_20190227T083207.SAFE/GRANULE/L2A_T48PYU_A019234_20190227T031435/IMG_DATA/R10m/T48PYU_20190227T030651_'
band2 = rasterio.open(imagePath+'B02_10m.jp2', driver='JP2OpenJPEG') #blue
band3 = rasterio.open(imagePath+'B03_10m.jp2', driver='JP2OpenJPEG') #green
band4 = rasterio.open(imagePath+'B04_10m.jp2', driver='JP2OpenJPEG') #red
band8 = rasterio.open(imagePath+'B08_10m.jp2', driver='JP2OpenJPEG') #near impact

In [4]:
def split_band(band):
    """Split une bande de Sentinel-2 en bande plus petite
    2 916 images de 20x20"""
    result = []
    my_band = band.read(1)
    for x in range(IMAGE_WIDTH):
        columns = []
        for y in range(IMAGE_WIDTH):
            lines = []
            for i in range(IMAGE_PIXEL):
                cells = []
                for j in range(IMAGE_PIXEL):
                    cells.append(my_band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

In [5]:
def create_smaller_images(blue, green, red, infra_red, rgb_name, rir_name):
    """Créée des images plus petites pouvant être mieux traitées par le réseau de neurones"""
    smaller_blue = split_band(blue)
    print("Fin bande bleue")
    smaller_green = split_band(green)
    print("Fin bande verte")
    smaller_red = split_band(red)
    print("Fin bande rouge")
    smaller_infra_red = split_band(infra_red)
    print("Fin bande infra rouge")
    for x in range(IMAGE_WIDTH):
        for y in range(IMAGE_WIDTH):
            chang = blue.transform * (y * IMAGE_PIXEL,x * IMAGE_PIXEL) #Pas trouvé mieux pour l'instant
            transform = affine.Affine(10.0, 0, chang[0], 0.0, -10.0, chang[1])
            smaller_image_rgb = rasterio.open(rgb_name + '_' + str(x) + '_' + str(y) + '.tiff', 'w', driver='Gtiff', 
                          width=IMAGE_PIXEL, height=IMAGE_PIXEL, count=3, crs=blue.crs, 
                          transform=transform, 
                          dtype=blue.dtypes[0]
                         )
            smaller_image_rgb.write(smaller_blue[x][y], 3)
            smaller_image_rgb.write(smaller_green[x][y], 2)
            smaller_image_rgb.write(smaller_red[x][y], 1)
            smaller_image_rgb.close()
            smaller_image_rir = rasterio.open(rir_name + '_' + str(x) + '_' + str(y) + '.tiff', 'w', driver='Gtiff', 
                          width=IMAGE_PIXEL, height=IMAGE_PIXEL, count=3, crs=red.crs, 
                          transform=transform, 
                          dtype=red.dtypes[0]
                         )
            smaller_image_rir.write(smaller_green[x][y], 3)
            smaller_image_rir.write(smaller_red[x][y], 2)
            smaller_image_rir.write(smaller_infra_red[x][y], 1)
            smaller_image_rir.close()
    print("Fichiers tiff créés.")

In [6]:
create_smaller_images(band2, band3, band4, band8, "output/sais_pas_gauche/rgb/gauche", "output/cafe_gauche/rir/gauche")

In [4]:
petite_bande = rasterio.open('outputsaispas/sais_pas_gauche/rir/gauche_121_121.tiff', driver='Gtiff')
print(band8.read())
print(band4.read())
print(band3.read())
print(petite_bande.read())

[[[   0    0    0 ... 1462 1540 1746]
  [   0    0    0 ... 1354 1462 1664]
  [   0    0    0 ... 1466 1522 1899]
  ...
  [2620 2438 2396 ... 1692 1888 1810]
  [2912 2575 2532 ... 1779 2189 2218]
  [2654 2420 2486 ... 1785 2213 2363]]]
[[[   0    0    0 ...  739  805  862]
  [   0    0    0 ...  650  789  792]
  [   0    0    0 ...  653  928  773]
  ...
  [ 464  464  516 ... 1090  952  988]
  [ 455  459  440 ...  765  689  791]
  [ 444  464  453 ...  807  733  840]]]
[[[  0   0   0 ... 579 621 686]
  [  0   0   0 ... 543 598 609]
  [  0   0   0 ... 570 818 630]
  ...
  [638 619 638 ... 579 598 544]
  [653 631 610 ... 479 510 548]
  [615 614 626 ... 511 546 631]]]
[[[1529 1582 1577 ...  230  578  964]
  [1762 1799 1854 ...  210  270  490]
  [2099 2088 2070 ...  195  203  199]
  ...
  [2065 2064 2021 ... 1692 1888 1810]
  [1952 1933 1888 ... 1779 2189 2218]
  [1789 1768 1815 ... 1785 2213 2363]]

 [[1080 1098 1108 ...  697  733 1050]
  [1208 1238 1274 ...  738  698  778]
  [1368 1348 135